In [182]:
import pandas as pd
import wbgapi as wb
import numpy as np

In [183]:
df = wb.data.DataFrame('NY.GNP.PCAP.CD', time=range(2010,2023), skipBlanks =True, columns='series')

In [184]:
df

NY.GNP.PCAP.CD
economy time                  
ABW     YR2010         23390.0
        YR2011         23400.0
        YR2012         24440.0
        YR2013         25500.0
        YR2014         25790.0
...                        ...
ZWE     YR2017          1170.0
        YR2018          1550.0
        YR2019          1450.0
        YR2020          1460.0
        YR2021          1530.0

[2914 rows x 1 columns]

In [185]:
df = df.reset_index()

In [186]:
df

,economy,time,NY.GNP.PCAP.CD
0,ABW,YR2010,23390.0
1,ABW,YR2011,23400.0
2,ABW,YR2012,24440.0
3,ABW,YR2013,25500.0
4,ABW,YR2014,25790.0
...,...,...,...
2909,ZWE,YR2017,1170.0
2910,ZWE,YR2018,1550.0
2911,ZWE,YR2019,1450.0
2912,ZWE,YR2020,1460.0


In [187]:
df = df.pivot(index='economy', columns='time', values='NY.GNP.PCAP.CD')

In [188]:
df

time,YR2010,YR2011,YR2012,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021
economy,,,,,,,,,,,,
ABW,23390.000000,23400.000000,24440.000000,25500.000000,25790.000000,26180.000000,26650.000000,27720.000000,29030.000000,30000.000000,24590.000000,29390.000000
AFE,1434.561864,1570.278589,1679.256511,1711.725316,1691.540731,1574.778660,1456.026987,1440.565428,1452.233541,1493.732269,1386.163976,1453.744944
AFG,520.000000,550.000000,650.000000,680.000000,650.000000,610.000000,570.000000,540.000000,520.000000,530.000000,500.000000,390.000000
AFW,1555.142811,1590.321609,1763.846950,1923.711925,2096.815123,2004.978658,1771.335338,1587.397582,1604.628244,1663.335667,1644.359873,1707.394011
AGO,3220.000000,3360.000000,4060.000000,4650.000000,4800.000000,3880.000000,2450.000000,1980.000000,1870.000000,2040.000000,1740.000000,1710.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,3150.000000,3390.000000,3540.000000,3800.000000,3910.000000,3930.000000,3930.000000,3950.000000,4300.000000,4640.000000,4440.000000,5130.000000
YEM,1130.000000,1010.000000,1180.000000,1340.000000,1360.000000,1110.000000,1070.000000,1000.000000,840.000000,NaN,NaN,NaN
ZAF,6810.000000,7700.000000,8320.000000,8050.000000,7340.000000,6550.000000,5960.000000,5950.000000,6390.000000,6730.000000,6090.000000,6530.000000


In [189]:
df = df.rename_axis(None, axis=1)
df.index.names= ['iso3']
df.columns=df.columns.str[2:]

In [190]:
df

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
iso3,,,,,,,,,,,,
ABW,23390.000000,23400.000000,24440.000000,25500.000000,25790.000000,26180.000000,26650.000000,27720.000000,29030.000000,30000.000000,24590.000000,29390.000000
AFE,1434.561864,1570.278589,1679.256511,1711.725316,1691.540731,1574.778660,1456.026987,1440.565428,1452.233541,1493.732269,1386.163976,1453.744944
AFG,520.000000,550.000000,650.000000,680.000000,650.000000,610.000000,570.000000,540.000000,520.000000,530.000000,500.000000,390.000000
AFW,1555.142811,1590.321609,1763.846950,1923.711925,2096.815123,2004.978658,1771.335338,1587.397582,1604.628244,1663.335667,1644.359873,1707.394011
AGO,3220.000000,3360.000000,4060.000000,4650.000000,4800.000000,3880.000000,2450.000000,1980.000000,1870.000000,2040.000000,1740.000000,1710.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,3150.000000,3390.000000,3540.000000,3800.000000,3910.000000,3930.000000,3930.000000,3950.000000,4300.000000,4640.000000,4440.000000,5130.000000
YEM,1130.000000,1010.000000,1180.000000,1340.000000,1360.000000,1110.000000,1070.000000,1000.000000,840.000000,NaN,NaN,NaN
ZAF,6810.000000,7700.000000,8320.000000,8050.000000,7340.000000,6550.000000,5960.000000,5950.000000,6390.000000,6730.000000,6090.000000,6530.000000


In [191]:
df = df.apply(pd.to_numeric, errors='coerce')

In [192]:
df.isna().sum()

2010     9
2011     8
2012     8
2013     6
2014     5
2015     4
2016     5
2017     4
2018     4
2019     5
2020    10
2021    18
dtype: int64

In [193]:
df = df.fillna(0).astype(np.int64, errors='ignore')
df = df.astype('int64')

In [194]:
df = df.round(decimals=0)

In [195]:
df.dtypes

2010    int64
2011    int64
2012    int64
2013    int64
2014    int64
2015    int64
2016    int64
2017    int64
2018    int64
2019    int64
2020    int64
2021    int64
dtype: object

In [196]:
df

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
iso3,,,,,,,,,,,,
ABW,23390,23400,24440,25500,25790,26180,26650,27720,29030,30000,24590,29390
AFE,1434,1570,1679,1711,1691,1574,1456,1440,1452,1493,1386,1453
AFG,520,550,650,680,650,610,570,540,520,530,500,390
AFW,1555,1590,1763,1923,2096,2004,1771,1587,1604,1663,1644,1707
AGO,3220,3360,4060,4650,4800,3880,2450,1980,1870,2040,1740,1710
...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,3150,3390,3540,3800,3910,3930,3930,3950,4300,4640,4440,5130
YEM,1130,1010,1180,1340,1360,1110,1070,1000,840,0,0,0
ZAF,6810,7700,8320,8050,7340,6550,5960,5950,6390,6730,6090,6530


In [197]:
df = df.reset_index()

In [198]:
df

,iso3,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ABW,23390,23400,24440,25500,25790,26180,26650,27720,29030,30000,24590,29390
1,AFE,1434,1570,1679,1711,1691,1574,1456,1440,1452,1493,1386,1453
2,AFG,520,550,650,680,650,610,570,540,520,530,500,390
3,AFW,1555,1590,1763,1923,2096,2004,1771,1587,1604,1663,1644,1707
4,AGO,3220,3360,4060,4650,4800,3880,2450,1980,1870,2040,1740,1710
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,XKX,3150,3390,3540,3800,3910,3930,3930,3950,4300,4640,4440,5130
246,YEM,1130,1010,1180,1340,1360,1110,1070,1000,840,0,0,0
247,ZAF,6810,7700,8320,8050,7340,6550,5960,5950,6390,6730,6090,6530
248,ZMB,1330,1390,1660,1720,1760,1540,1340,1270,1400,1390,1130,1030


In [199]:
df = pd.melt(df, id_vars = 'iso3', var_name='year', value_name='gnipc')

In [200]:
df

,iso3,year,gnipc
0,ABW,2010,23390
1,AFE,2010,1434
2,AFG,2010,520
3,AFW,2010,1555
4,AGO,2010,3220
...,...,...,...
2995,XKX,2021,5130
2996,YEM,2021,0
2997,ZAF,2021,6530
2998,ZMB,2021,1030


In [201]:
df = df.set_index(['iso3', 'year'])

In [202]:
df

,,gnipc
iso3,year,
ABW,2010,23390
AFE,2010,1434
AFG,2010,520
AFW,2010,1555
AGO,2010,3220
...,...,...
XKX,2021,5130
YEM,2021,0
ZAF,2021,6530
